### FE545 - Homework #2

**Author**: Sid Bhatia

**Date**: March 5th, 2023

**Pledge**: I pledge my honor that I have abided by the Stevens Honor System.

**Professor**: Steve Yang

An **Asian option** is a type of *exotic option*. Unlike a vanilla European option where the price of the option is dependent upon the price of the underlying asset at expiry, an Asian option pay-off is a function of multiple points up to and including the price at expiry. Thus it is “path-dependent” as the price relies on knowing how the underlying behaved at certain points before expiry. Asian options in particular base their price off the mean average price of these sampled points. To simplify the problem, we will consider equally distributed sample points beginning at time and ending at maturity,

In this problem, we will consider Geometric mean $A$ of the spot prices we use the following formula:

$$
A(0, T) = \exp{\left( \frac{1}{N} \sum_{i = 1}^N \log(S(t_i)) \right)}
$$

Unlike in the vanilla European option Monte Carlo case we have learned in class, where we only needed to generate multiple spot values at expiry, we now need to generate multiple spot paths, each sampled at the correct points. Thus, instead of providing a double value representing spot to our option, we now need to provide a $\texttt{std::vector<double>}$ (i.e. a vector of double values), each element of which represents a sample of the spot price on a particular path. We will still be modeling our asset price path via a Geometric Brownian Motion (GBM), and we will create each path by adding the correct drift and variance at each step in order to maintain the properties of GBM.

Implement **PayOff classes** according to the following template:

```cpp
#ifndef __PAY_OFF__
#define __PAY_OFF__

#include <algorithm> // This is needed for the std::max  comparison function, used in the pay-off calculations

class PayOff {
    public:
        PayOff(); // Default (no parameter) constructor
        virtual ~PayOff() {}; // Virtual destructor
        // Overloaded () operator, turns the PayOff into an abstract
        // function object
        virtual double operator(const double& S) const = 0;
};

class PayOffCall : public PayOff {
    public:
        PayOffCall(const double& K_);
        virtual ~PayOffCall() {};
        // Virtual function is now over-ridden (not pure-virtual anymore)
        virtual double operator(const double& S) const;

    private:
        double K; // Strike price  
};

class PayOffPut : public PayOff {
    public:
        PayOffPut(const double& K_);
        virtual ~PayOffPut() {};
        virtual double operator(const double& S) const;
    private:
        double K; // Strike
};

#endif  
```